---
### 9.3.1 参数估计
假设总体的概率分布类型已知，但其中含有未知参数，由样本估计参数的的值

样本均值的标准误差 (SEM)：${\rm SEM}=\dfrac{s}{\sqrt n}$

In [1]:
import numpy as np
from scipy.stats import t, sem

d = np.loadtxt('../../09第9章  数据的描述性统计方法/data9_11.txt').flatten()
n = len(d)
mu = np.mean(d)
sig = np.std(d, ddof=1)
SEM = sem(d)
a = 0.05    # 显著性水平 = 1 - 置信水平
ta = t.ppf(1-a/2, n-1)
L = [mu - SEM*ta, mu + SEM*ta]
L   # 总体均值 mu 的一个置信水平为 1-a 的置信区间

[500.44510746243924, 507.05489253756076]

### 9.3.2 参数假设检验
总体的分布函数形式已知，但其中含有未知参数，为了推断总体的某些性质，提出某些关于总体参数的假设，然后根据样本对所提出的假设作出判断，是接受还是拒绝

#### 1. 单个总体均值的假设检验
1）正态总体标准差 $\sigma$ 已知的 $Z$ 检验法

检验假设
$$H_0:\mu=26, \quad H_1:\mu\neq 26$$

In [2]:
from statsmodels.stats.weightstats import ztest
from scipy.stats import norm

alpha, sigma = 0.05, 4.2
data = np.array([26.01, 26.00, 25.98, 25.86, 26.32, 25.58, 25.32, 25.89, 26.32, 26.18])
std_d = 26
tstat, pvalue = ztest(data, value=std_d)

mu = data.mean()
sig = data.std(ddof=1)
z = tstat*sig/sigma
za = norm.ppf(1-alpha/2, 0, 1)  # 标准正态分布
print('Z统计量值：', z); print('p值：', pvalue); print('分位数：', za)

Z统计量值： -0.04065785563073785
p值： 0.5847910057871308
分位数： 1.959963984540054


**结果的两种解释**

利用检验统计量的解释：因为 $|z|<z_{\alpha/2}$，未落入拒绝域，故在显著性水平 $\alpha=0.05$ 下接受零假设 $H_0$，即认为总体均值 $\mu=26$，纽扣总体的直径符合标准

$p$ 值解释：p-value 是一种概率，表示在原假设为真的前提下，出现该样本或比该样本更极端的结果的概率之和。如果 $p$ 值很小，说明在原假设下极端观测结果的发生概率很小。而它又出现了，根据小概率原理，就有理由拒绝原假设；$p$ 值越小，拒绝原假设的理由越充分。$p$ 值小于 $0.05$ 被认为是确定实验数据可靠性的金标准。这里 $p$ 值约为 $0.5848$，远大于选定的显著水平 $0.05$，所以有 $95\%$ 的把握接受 $H_0$ 而不会犯错

2）正态总体标准差 $\sigma$ 未知的 $t$ 检验法

检验假设
$$H_0:\mu\leq225,\quad H_1:\mu>225$$

In [3]:
from scipy.stats import ttest_1samp

data = np.loadtxt('../../09第9章  数据的描述性统计方法/data9_13.txt').flatten()
mu = data.mean()
sig = data.std(ddof=1)
n = len(data)
ta = t.ppf(0.95, n - 1)
ts, p = ttest_1samp(data, 225, alternative='greater')   # 备选假设 1 大于 value 值 225
print('t统计量值：', ts); print('p值：', p); print('分位数：', ta)

t统计量值： 0.6685176967463559
p值： 0.2569800715875837
分位数： 1.7530503556925547


接受 $H_0$

#### 2. 两个正态总体均值差的 $t$ 检验
检验假设
$$H_0:\mu_1-\mu_2\leq0,\quad H_1:\mu_1-\mu_2>0$$

In [4]:
import pandas as pd
from scipy.stats import t
from statsmodels.stats.weightstats import ttest_ind

df = pd.read_csv('../../09第9章  数据的描述性统计方法/data9_14.txt', sep='  ', header=None, engine='python')
data_a, data_b = df.values
data_b = data_b[~np.isnan(data_b)]
ts, p, df = ttest_ind(data_a, data_b, alternative='larger')
print('检验统计量为：', tstat); print('p值为:', p); print('自由度：', df)

检验统计量为： -0.5464000062893251
p值为: 0.0012755021070520822
自由度： 19.0


### 9.3.3 非参数假设检验
在实际建模中，我们对样本数据服从什么分布往往是完全未知的，这就需要进行非参数假设检验
#### 1. 分布拟合的卡方检验
1）检验假设

$H_0:X {\text的分布律为}$
$$P\{X=k\}=\dfrac{\lambda^k{\rm e}^{-\lambda}}{k!}\tag{\text 泊松分布}$$

In [5]:
from scipy.stats import poisson, chisquare, chi2

n1 = np.array([36, 40, 19, 2, 0, 2, 1])
f = np.arange(7)
n = n1.sum()
_lambda = n1 @ f / n
p0 = poisson.pmf(range(3), _lambda)
p = np.hstack([p0, 1-sum(p0)])
exp_freq = n*p
obs_freq = np.hstack([n1[:3], sum(n1[3:])])
obs_freq, exp_freq

(array([36, 40, 19,  5]),
 array([36.78794412, 36.78794412, 18.39397206,  8.03013971]))

In [6]:
chis1 = chisquare(obs_freq, exp_freq, ddof=1).statistic     # 两种方法计算统计量
chis2 = sum(obs_freq**2 / (exp_freq)) - n                   # ~

k = len(obs_freq)
r = 1
chisa = chi2.ppf(0.95, k-r-1)
print(f'{chis1 = }\n{chis2 = }')
print(f'{chisa = }')

chis1 = 1.4607074196299568
chis2 = 1.460707419629955
chisa = 5.991464547107979


$\chi^2 < \chi_{0.05}^2(k-r-1)$，接受 $H_0$，即认为样本来自泊松分布的总体

2）检验假设

$H_0:灯泡寿命服从指数分布$
$$f(t)=\begin{cases}0.005{\rm e}^{-0.005t},\quad &t\geq0\\0,\quad &t<0 \end{cases}$$

In [7]:
import scipy.stats as ss
t = np.array([100, 200, 300, np.inf])
p0 = ss.expon.cdf(t, scale=200)     # theta = 1/0.005 = 200
p = np.hstack([p0[0], np.diff(p0)])

obs_freq = np.array([121, 78, 43, 58])
n = sum(obs_freq)
exp_freq = p*n
obs_freq, exp_freq

(array([121,  78,  43,  58]),
 array([118.04080209,  71.59536556,  43.42478431,  66.93904804]))

In [8]:
chis, p = ss.chisquare(obs_freq, exp_freq)
k = len(obs_freq)
r = 0
chisa = ss.chi2.ppf(0.95, k-r-1)
print(f'{chis  = }')
print(f'{chisa = }')

chis  = 1.8449946436260816
chisa = 7.814727903251179


$\chi^2 < \chi_{0.05}^2(k-r-1)$，故在显著性水平 $\alpha=0.05$ 下，接受 $H_0$，即认为这批灯泡寿命服从指数分布

#### 2. 柯尔莫哥洛夫 (Kolmogorov-Smirnov) 检验
直接针对原假设 $H_0:F(x)=F_0(x)$ 检验分布函数的正确性，这里的分布函数 $F(x)$ 必须是连续型分布

检验假设 $H_0:X \thicksim N\left(\mu, \sigma^2\right)$

In [9]:
df = pd.read_csv('../../09第9章  数据的描述性统计方法/data9_17.txt', sep='  ', header=None, engine='python')
data = df.values.flatten()
data = data[~np.isnan(data)]
mu = data.mean()
sig = data.std(ddof=1)
st, p = ss.kstest(data, 'norm', (mu, sig))
print("统计量：", st)
print("p值：", p)

统计量： 0.08509761671300922
p值： 0.5484251821065218


$D_n=0.0851<1.36/\sqrt{n}$，故接受 $H_0$